<a href="https://colab.research.google.com/github/Andreylive/UAV-human-detection/blob/main/YOLOv5_(medium%2C_not_pretrained).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!git pull

!git reset --hard 886f1c03d839575afecb059accf74296fad395b6

In [ ]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.google_utils import gdrive_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
!curl -L "https://app.roboflow.com/ds/YCsKLvAbzh?key=wauxyPlLFl" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat data.yaml

In [ ]:
# define number of classes based on YAML
import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
#this is the model configuration we will use for our tutorial 
%cat ./data.yaml

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate ./data.yaml

# parameters
train : ./train
val : ./valid
names : ['Person']
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
%cat ./models/yolov5m.yaml

In [ ]:
!pip install wandb -q

In [ ]:
cd yolov5

In [ ]:
ls

In [ ]:
!git pull

In [ ]:
if torch.cuda.is_available():
    print("GPU is available")
    device = torch.device("cuda:0")
else:
    print("GPU not available, using CPU instead")
    device = torch.device("cpu")


In [ ]:
!export WANDB_API_KEY=742aac2df6aee3508d1fdb5ad95cb60e7ea2dbdd && python train.py --img 416 --batch 16 --epochs 100 --data './data.yaml' --cfg ./models/yolov5m.yaml --weights '' --name ./yolov5m_results --cache

In [ ]:
img = mpimg.imread('./runs/train/yolov5s_results2/results.png')

plt.figure(figsize=(20, 10))
plt.imshow(img)
plt.show()

In [ ]:
!python detect.py --weights ./runs/train/yolov5s_results2/weights/best.pt --img 416 --conf 0.4 --source ./valid/images

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
for det_img in [
    'train_BLA_0006_JPG.rf.586dd34289f6bc89c3fd33b16c30190e.jpg',
    'train_BRA_1003_JPG.rf.a3f32a60314fdb79d054f284d712f1c8.jpg',
    'train_ZRI_2012_JPG.rf.c9d7bfac5fbc26c34c2e567ee09ed736.jpg',
    'train_ZRI_2018_JPG.rf.20a15d8fc0c97f8f4add326d3890030c.jpg']:
    # Загрузка изображения
    img = mpimg.imread('/kaggle/working/yolov5/runs/detect/exp6/'+det_img)

    # Отображение изображения
    plt.imshow(img)
    plt.show()

In [ ]:
!python val.py --weights ./runs/train/yolov5m_results2/weights/best.pt --data ./data.yaml --img 416 --half

In [ ]:
!ls /runs/val/exp

In [ ]:
img = mpimg.imread('./runs/val/exp/confusion_matrix.png')

plt.figure(figsize=(20, 10))
plt.imshow(img)
plt.show()

#### FPS
FPS означает «Кадров в секунду». Это метрика, обычно используемая для оценки производительности приложений компьютерного зрения в реальном времени, таких как обнаружение объектов, отслеживание и обработка видео. В контексте моделей обнаружения объектов и глубокого обучения, таких как YOLO, FPS используется для измерения того, насколько быстро модель может обрабатывать изображения на этапе вывода или прогнозирования, а не на этапе обучения.

Более высокий FPS указывает на то, что модель может обрабатывать больше изображений за заданный промежуток времени, что делает ее более подходящей для приложений реального времени, где требуется быстрое и точное обнаружение. Более низкие значения FPS могут быть не идеальными для сценариев реального времени, но все же могут использоваться в менее чувствительных ко времени приложениях.

Посчитаем его основываясь на параметрах детекции:

    - Pre-process time: 0.3ms
    - Inference time: 11.2ms
    - NMS (Non-Maximum Suppression) time: 0.3ms
    
Чтобы рассчитать FPS, нужно сложить эти времена вместе, чтобы получить общее время на изображение, а затем преобразовать его в кадры в секунду.

Общее время на изображение (в секундах) = (0,3 + 11,2 + 0,3) / 1000 = 0,0118 секунд

Теперь посчитаем FPS:

FPS = 1 / Общее время на изображение = 1 / 0,0118 = 84,75

Итак, FPS для вашей модели составляет 84,75.

## Итог
Мы использовали модель Yolo V5 small
Это настраиваемый файл конфигурации модели YOLOv5. Модель имеет конструкцию, состоящую из позвоночника и головы. Магистраль отвечает за извлечение признаков, а головка отвечает за обнаружение объектов с помощью блоков привязки. Вот разбивка структуры:

 train и val указывают пути набора данных для обучения и проверки соответственно.
 имена содержат имена классов в вашем наборе данных (в данном случае «Человек»).
 nc — количество классов.
 depth_multiple и width_multiple — это гиперпараметры, управляющие глубиной и шириной модели.
 привязки — это предопределенные размеры блока привязки для различных масштабов карты объектов (P3/8, P4/16, P5/32).

Костяк состоит из:

Focus layer: объединяет пространственную информацию от соседних пикселей во входном изображении.
Conv layers: выполните свертки с шагом 2, чтобы уменьшить выборку карт объектов.
BottleneckCSP layers: межэтапные иерархические сети для лучшего градиентного потока и повторного использования функций.
SPP (Spatial Pyramid Pooling) layer: объединяет объекты в нескольких масштабах для лучшего обнаружения объектов в разных масштабах.

head состоит из:

 Слои Conv: уменьшите размер канала карт объектов.
 Повышение дискретизации слоев: повышайте дискретизацию карт объектов для объединения с объектами более низкого уровня.
 Слои Concat: объединяйте карты объектов из основы и головы.
 Слои узких мест CSP: Уточните объединенные карты объектов.
 Слой обнаружения: выполните обнаружение объектов с помощью полей привязки и выведите окончательные прогнозы.

Модель использует три разных масштаба карты признаков (P3/8, P4/16, P5/32) для обнаружения объектов, что помогает обнаруживать объекты разных размеров на входных изображениях.

## метрики
- P    0.33          
- R    0.221    
- mAP50 0.166 
- FPS 84,75

In [ ]:
import shutil

shutil.move('source_path/filename.extension', '/kaggle/working/filename.extension')


In [ ]:
!tar -czf output.tar.gz /kaggle/working/*
